<style>
h1, h2, h3 {
    text-align: center;
    font-weight: bold;
}
</style>


# NYC Toilet Pattern - Behind The Site

<div style="text-align: center">
<i>Li, Junrui & Fu, Tongzheng<br />
Group <b>69</b><br />
</i>
</div>


<style>
h1, h2, h3 {
    text-align: center;
    font-weight: bold;
}
</style>

## 1. Motivation

<style>
h1, h2, h3 {
    text-align: center;
    font-weight: bold;
}
</style>


### Dataset Overview

|Dataset name|Tags|URL|Comments|
|:---|:---:|:---:|---|
|NYC Public Restrooms Distribution|csv, geo|[url](https://data.cityofnewyork.us/City-Government/Public-Restrooms-Operational-/vzrx-zg6z)|Base dataset, useful columns like: toilet distrubution geo info, opening hours, operator etc.|
|NYC Pedestrian Demand Map|csv, geo|[url](https://data.cityofnewyork.us/Transportation/Pedestrian-Mobility-Plan-Pedestrian-Demand-Map/c4kr-96ik)|From a transportation perspective, public toilets are most relevant to pedestrians compared to private cars or public transit etc. We'll use this dataset to explore the relationship between toilet distribution and pedestrian walkway intensity.|
|NYC Points of Interest|csv, geo|[url](https://data.cityofnewyork.us/City-Government/Points-of-Interest/rxuy-2muj)|From a perspective of city planning, we use this dataset to explore whether public toilet distribution shows similarities with specific functional locations.|

### Goal -- heavily user-driven

Target reader:
- Society Scholer
- User that has some kind of toilet-anxious like me(Junrui)
- Bored people like me(Junrui)

Thus, our goal is: create a comfortable visual and browsing experience, provide good interaction with user space as much as we could offer. (Let the users explore themselves)



<style>
h1, h2, h3 {
    text-align: center;
    font-weight: bold;
}
h4{
    font-weight: bold;
}
</style>

## **2. Basic Stats**

<style>
h1, h2, h3 {
    text-align: center;
    font-weight: bold;
}
h4{
    font-weight: bold;
}
</style>

### Data Pre-processing

> Here I discuss two notable pre-processing steps. 
> 
> Some basic data processing workflow that ensures correctness, such as regular expression matching, handling NaN value filtering are not included.
> 
>

#### LLM-based Semantic Processing

This is when we are doing the temporal visualizaiton with public restroom's opening hours.

The original operating hours data had inconsistent formats that were difficult to standardize with traditional regular expressions. Shown in following code cell:

In [ ]:
import pandas as pd
import plotly.express as px

RESTROOM_DATA_PATH = './data/Public_Restrooms.csv'
df_toilet_dis = pd.read_csv(RESTROOM_DATA_PATH)
df_toilet_dis.iloc[[1,6,4]][['Facility Name', 'Open','Hours of Operation','Location']]

,Facility Name,Open,Hours of Operation,Location
1,Passerelle Building,Year Round,"8am-4pm, Open later seasonally",POINT (-73.843252 40.751662)
6,55 East 52nd Street POPS,Year Round,Everyday 8:00 am-10:00 pm,POINT (-73.973672 40.759132)
4,"Bushwick Library, BPL",Year Round,Monday\t10 am - 6 pm\nTuesday\t1 pm - 8 pm\nWe...,POINT (-73.93962 40.704567)


From the table, we can see that the "Hours of Operation" column has many different formatting styles, making traditional regex matching and formatting very troublesome. 

Thus, we choose to have an LLM help us process this information, utilizing its semantic understanding capabilities to standardize the time formats. The results are shown below.

In [29]:
RESTROOM_DATA_PATH_TIME = './data/Public_Restrooms_time.csv'
df_toilet_dis_unified = pd.read_csv(RESTROOM_DATA_PATH_TIME)
df_toilet_dis_unified.iloc[[1,6,4]]

,Facility Name,Operator,Status,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
1,Passerelle Building,NYC Parks,Operational,08:00-16:00,08:00-16:00,08:00-16:00,08:00-16:00,08:00-16:00,08:00-16:00,08:00-16:00
6,55 East 52nd Street POPS,Park Avenue Plaza Owner LLC,Operational,08:00-22:00,08:00-22:00,08:00-22:00,08:00-22:00,08:00-22:00,08:00-22:00,08:00-22:00
4,"Bushwick Library, BPL",BPL,Operational,10:00-18:00,13:00-20:00,10:00-18:00,10:00-20:00,10:00-18:00,10:00-17:00,CLOSED


#### **- WKT to Points**
In our NYC Pedestrian Demand Map dataset, the original dataset records the geographic information of each NYC street through WKT format in the column "the_geom".


> WKT Introduction<br>
> 
> WKT (Well-Known Text) is a text markup language used to represent geospatial data, providing a standardized way to describe geometric objects. Common WKT format types include:
> 
> - POINT: ```POINT (30 10)```
> - LINESTRING: ```LINESTRING (30 10, 10 30, 40 40)```
> - POLYGON: ```POLYGON ((30 10, 40 40, 20 40, 10 20, 30 10))```
> - MULTILINESTRING ```MULTILINESTRING ((10 10, 20 20, 10 40), (40 40, 30 30, 40 20, 30 10))```
> 

In [35]:
RESTROOM_DATA_PATH_TIME = './data/Pedestrian_Mobility_Plan_Pedestrian_Demand_20250425.csv'
df_pedastrain = pd.read_csv(RESTROOM_DATA_PATH_TIME)
print("The dataset length: ", len(df_pedastrain))
df_pedastrain.iloc[[0]][["the_geom","street","Rank","SHAPE_Leng"]]

The dataset length:  127277


,the_geom,street,Rank,SHAPE_Leng
0,MULTILINESTRING ((-73.81665096623519 40.674537...,122 STREET,4,326.224892


Here, the raw dataset uses MULTILINESTRING to describe street information. 

Because the dataset is very detailed and large, with 127,277 rows, we cannot directly render a map like the one shown at [url](https://data.cityofnewyork.us/Transportation/Pedestrian-Mobility-Plan-Pedestrian-Demand-Map/c4kr-96ik) locally where my computer would stuck. 

To reduce the data to a level that can be processed locally, we perform "point sampling" on the multi lines of whith the format could be fit into ploty python hexbin api. 

The main process goes:
1. determine number of points based on the lin length(column SHAPE_Leng)
2. sample points along the line evenly

The **pseudocode** for this operation is as follows:

```python 
points = []

for each row in lineDataGDF:
   # Reverse Rank value so smaller Rank corresponds to higher intensity
   intensity = 6 - row["Rank"]
   
   geom = current row's geometry object
   
   # Determine number of points based on line length
   # Longer lines contribute more points, minimum 3 points
   numberOfPoints = max(3, floor(row["length_m"] / 50))  # Extract 1 point per 50 meters, minimum 3 points
   
   for each line in geom:
       sample points evenly along the line
       
       # Interpolate using normalized position on the line
       POINT = interpolate at position i/(numberOfPoints-1) on line
       add POINT to points[]

return DataFrame(points[])
```

<style>
h1, h2, h3 {
    text-align: center;
    font-weight: bold;
}
h4{
    font-weight: bold;
}
</style>

## 3. Data Analysis

<style>
h1, h2, h3 {
    text-align: center;
    font-weight: bold;
}
h4{
    font-weight: bold;
}
</style>


First, I want to divide my data processing-analysis for this project into two categories:

```
Category 1. Raw data --- --- --- ---> Visualization ---> Analysis
Category 2. Raw data ---> Mining ---> Visualization ---> Analysis

Category 1
   Map 1 - NYC pedastrain intensity and toilets distribution map
   Map 2 - NYC public toilet service coverage map
   Solar bar chart - NYC public toilets opening hours clock

Category 2
   Heat matrix - Affinity heat matrix between NYC public toilets and other domain categories
```

For `Category 1`, in our project, all of its figures are for **reader-driven** which means the main task for us here is to complete the visualization of the Raw data, plot them nice.

Thus, here, I will focus on explaining the `Mining` part we did in `Category 2` with the heat matrix, which better fits my understanding of data analysis that should be included in the current Jupyter document (just like the Machine learning mentioned in the project requirement, which is a part of `Mining`).

### Task Overview

For the heap matrix presented in the website

During the mining prosess, I explored the spatial distribution similarity between public restrooms and various Points of Interest (POI) in New York City. The focus was on determining which types of urban facilities share similar geographic distribution patterns with public restrooms.

#### Dataset Components

The analysis relied on two key data attributes:

1. **Geographic coordinates**: Latitude and longitude of each facility
2. **Category labels**: Facility type classifications including:
   - Public Restrooms
   - Educational Facilities
   - Cultural Facilities
   - Recreational Facilities
   - Social Services
   - Transportation Facilities
   - ......

#### Visualization Options

To analyze the distribution affinity, several visualization approaches were considered:

1. **Spatial density maps**: Direct visualization of each facility type's distribution
2. **MDS plots**: 2D projection preserving distance relationships between facility types
3. **t-SNE plots**: Nonlinear dimensionality reduction emphasizing local clustering patterns
4. **Similarity heat matrix**: Direct visualization of similarity values between all facility pairs

The similarity heat matrix was selected as the most user-friendly and intuitive visualization method because it:

- Directly shows similarity values between all facility types
- Allows quick identification of facilities most similar to public restrooms
- Provides exact numerical similarity values via tooltips
- Presents a clean, easily interpretable format for non-technical audiences

### Mathematical Methodology

The heat matrix visualization represents spatial distribution similarity between facility types in NYC, focusing specifically on these key methodological components:

#### Spatial Density Grid Analysis
- Created uniform grid cells (0.01° × 0.01°) across NYC
- Generated 2D histograms of facility occurrences within each grid cell
- Normalized density distributions by total facility count per type

#### Cosine Similarity Metric
The core mathematical method behind the heat matrix is the cosine similarity measure:

$$\text{similarity}(A,B) = \frac{A \cdot B}{||A|| \cdot ||B||} = \frac{\sum_{i=1}^{n} A_i B_i}{\sqrt{\sum_{i=1}^{n} A_i^2} \sqrt{\sum_{i=1}^{n} B_i^2}}$$

Where:
- A and B are spatial density vectors for different facility types
- Each vector component represents normalized facility density in a specific grid cell
- The measure quantifies similarity in spatial distribution patterns regardless of absolute facility counts

This approach enables comparison of spatial patterns rather than raw facility numbers, identifying meaningful relationships between different urban facility distributions.

### Implementation

The heat matrix visualization was implemented using:

- **NumPy's histogram2d**: For computing spatial density grids
- **Scikit-learn's cosine_similarity**: For calculating similarity between facility type distributions

The reletad part of my code is presented in the following code cell.

In [3]:
def create_spatial_density_grid(df, lat_min, lat_max, lon_min, lon_max, grid_size=0.01):
    """
    Create spatial density grid for each facility type
    """
    # Create grid
    lat_bins = np.arange(lat_min, lat_max + grid_size, grid_size)
    lon_bins = np.arange(lon_min, lon_max + grid_size, grid_size)
    
    # Initialize result dictionary
    density_map = {}
    
    # Get unique facility types
    facility_types = df['FACILITY_T'].unique()
    
    # Create density grid for each facility type
    for facility_type in facility_types:
        # Filter facilities of this type
        subset = df[df['FACILITY_T'] == facility_type]
        
        # Ensure sufficient data points
        if len(subset) < 2:
            print(f"Warning: Facility type {facility_type} has only {len(subset)} data points, cannot create valid density grid. Skipping.")
            continue
        
        try:
            # Create 2D histogram
            hist, _, _ = np.histogram2d(
                subset['Latitude'], 
                subset['Longitude'], 
                bins=[lat_bins, lon_bins]
            )
            
            # Normalize density to balance differences in facility counts
            if hist.sum() > 0:  # Avoid division by zero
                hist = hist / hist.sum()
                
            # Flatten histogram as feature vector
            density_map[facility_type] = hist.flatten()
        except Exception as e:
            print(f"Error processing facility type {facility_type}: {e}")
    
    return density_map

def compute_similarity_matrix(density_map):
    """
    Compute cosine similarity matrix between facility types
    """
    # Get all facility types
    facility_types = list(density_map.keys())
    
    # Create feature matrix, each row is a density vector for a facility type
    features = np.array([density_map[ft] for ft in facility_types])
    
    # Compute cosine similarity
    similarity_matrix = cosine_similarity(features)
    
    return similarity_matrix, facility_types

<style>
h1, h2, h3 {
    text-align: center;
    font-weight: bold;
}
h4{
    font-weight: bold;
}
</style>

## 4. Genre

<style>
h1, h2, h3 {
    text-align: center;
    font-weight: bold;
}
h4{
    font-weight: bold;
}
</style>


### Genre Strategy

<div align="center">

| Type | Comments |
|------|----------|
| **Magazine Style** | Enhances readability and engagement |
| **Reader-Driven** | Emphasizes interactivity, enables user-led exploration, supports personalized data discovery, accommodates diverse user backgrounds |

</div>

### Visual Narrative

<div align="center">

| Category | Tool | Comments |
|----------|------|----------|
| **Visual Structuring** | Establishing Shot | Top map provides comprehensive geographic overview, establishing spatial context |
|  | Consistent Visual Platform | Maintains uniform design language, typography and color scheme across visualizations |
| **Highlighting** | Feature Distinction | Color-coding restroom status (operational/non-operational) enables quick pattern recognition |
| **Transition Guidance** | Object Continuity | Consistent color encoding across visualizations maintains cognitive flow |
|  | Familiar Objects | Map-based visualization leverages users' geographic literacy, reducing learning curve |

</div>

### Narrative Structure

<div align="center">

| Category | Tool | Comments |
|----------|------|----------|
| **Ordering** | User Directed Path | Empowers users to determine exploration sequence and depth |
|  | Random Access | Provides flexible navigation suitable for diverse user interests |
| **Interactivity** | Hover Highlighting | Delivers contextual details without disrupting overall visualization flow |
|  | Filtering/Selection | Radius selector and status filters enable focused analysis of specific subsets |
|  | Stimulating Default Views | Engaging initial visualizations spark curiosity and provide meaningful overviews |
| **Messaging** | Captions/Headlines | Clear titles contextualize each visualization's purpose |
|  | Annotations | Legends and markers ensure accurate interpretation of visual encodings |
|  | Figure Labels | Structural labeling (e.g., "Fig. 3: Affinity heat matrix") creates visual hierarchy |

</div>


<style>
h1, h2, h3 {
    text-align: center;
    font-weight: bold;
}
h4{
    font-weight: bold;
}
</style>

## **5. Visualizations**

<style>
h1, h2, h3 {
    text-align: center;
    font-weight: bold;
}
h4{
    font-weight: bold;
}
</style>


### Why

From our visualization, we would like to solve the following questions

1. where they're located, 
2. when they're open, 
3. who manages them, 
4. what facilities are nearby.

### Who
Possible readers of field social scholar: urban planners (spatial distribution), facility managers (institutional responsibilities), and general public (accessibility and operating hours)

### How

Here our implementation goes into these two categories

- Spatial Dimension
	- **Distribution**: Pedestrian intensity and restroom location map (top)
	- **Accessibility**: Coverage radius map (middle)
	- **Correlation**: Restroom-domain category affinity heat matrix (lower middle)

- Temporal Dimension
	- **Availability**: 24-hour opening hours clock diagram (bottom)

**In detail:**

#### 1. NYC pedastrain intensity and toilets distribution map
- Visualizes spatial correlation between public toilets and pedastrain traffic
- Identifies service gaps in high-density pedestrian areas

#### 2. NYC public toilet service coverage map
- Reveals spatial accessibility and service "deserts"
- Enables scenario testing through radius adjustment
- Provides planning insights for new facility placement
- Assesses spatial equity of public servicess

#### 3. Affinity heat matrix between NYC public toilets and other domain categories
- Uncovers complex relationships between restrooms and other urban services
- Identifies service clustering patterns that may inform planning decisions
- Highlights potential cross-departmental collaboration opportunities

#### 4. NYC public toilets opening hours clock
- Visually represents restroom availability across different hours, helping users locate accessible facilities at specific times
- Highlights temporal service gaps, particularly during overnight or early morning hours
- Evaluates time-based equity of public services, especially for shift workers and non-traditional schedules

<style>
h1, h2, h3 {
    text-align: center;
    font-weight: bold;
}
h4{
    font-weight: bold;
}
</style>

## 6. Discussion

<style>
h1, h2, h3 {
    text-align: center;
    font-weight: bold;
}
h4{
    font-weight: bold;
}
</style>

### What Went Well

- Achieved high-quality spatial and temporal visualizations that meet our aesthetic standards
- Implemented diverse analytical content with multiple interactive tools
- Established technical foundation integrating Python and JavaScript when doing visualization
- The team collaboration goes really smooth

### Areas for Improvement

#### - Technical Limitations

##### Integration & Platform

Current visualizations exist as separate entities rather than a cohesive dashboard. We need a true partitioned poster approach with linked visualizations, like time-filtered coverage radius. Dashboard implementation faced challenges with Python libraries (Plotly Dash) due to customization limitations. More extensive HTML/JS/CSS integration would enhance flexibility but was constrained by project timeline. We are now considering responsive design.

##### Visualization Development

Python-based web visualization offers streamlined workflow but lacks customization flexibility. A hybrid approach was required, embedding JavaScript within Python, to achieve desired interactions. Web presentation context demands higher attention to visual and interactive experience than scientific visualization.

#### - Non-Technical Considerations

In the end we created a user-driven presentation for this project. Cuz we stuck at the very beginning for a while, maybe with more time, we could do more analysis and exploration from specific angles, balancing user-driven and author-guided approaches. 

For our's feeling, the current work could be like the first step of a larger project, possibly related to digital twins, social sciences or similar fields. During team discussions, we felt excited about the future possibilities of this project.

<style>
h1, h2, h3 {
    text-align: center;
    font-weight: bold;
}
h4{
    font-weight: bold;
}
</style>

## 7. Contributions

<h5 align="center"> <i>Group 69</i> Contribution Matrix</h5>

<div style="display: flex; justify-content: center;">


|Task|Li, Junrui|Fu, Tongzheng|
|:---|:---:|:---:|
|Diagram - Fig.1-sub scatter map||🔸|
|Diagram - Fig.1-sub hexbin map|🔸||
|Diagram - Fig.1 integration|🔹|🔹|
|Diagram - Fig.2 coverage map|🔸||
|Diagram - Fig.3 heat matrix|🔹|🔹|
|Diagram - Fig.4 clock-like polar bar chart|🔸||
|Content - web page ||🔸|
|Content - jupyter document|🔸||
|Formatting|🔹|🔹|

</div>

<div style="text-align: center;">
<br>

🔸 means mainly implemented by the team member.<br>
🔹 means kind of equal.
<br>
</div>

<style>
h1, h2, h3 {
    text-align: center;
    font-weight: bold;
}
h4{
    font-weight: bold;
}
</style>

## 8. Appendix [Source Code]

In [ ]:
#        toilet_scatter.py           
#####################################

import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def load_restroom_data(file_path):
    """
    Load public restroom data
    
    Args:
        file_path (str): Path to CSV file
    
    Returns:
        pandas.DataFrame: Loaded data
    """
    df = pd.read_csv(file_path)
    return df

def create_restroom_map(data_path, center_lat=40.7128, center_lon=-74.0060, zoom=10, height=600):
    """
    Create public restroom map visualization, including data loading and analysis
    
    Args:
        data_path (str): Path to CSV file
        center_lat (float): Map center latitude
        center_lon (float): Map center longitude
        zoom (int): Map zoom level
        height (int): Map height
    
    Returns:
        plotly.graph_objects.Figure: Created map figure
    """
    # Load data
    df = load_restroom_data(data_path)
    
    # Create map
    fig = px.scatter_mapbox(
        df, 
        lat="Latitude", 
        lon="Longitude",
        hover_name="Facility Name",
        hover_data={
            "Location Type": True,
            "Status": True,
            "Hours of Operation": False,
            "Accessibility": False,
            "Latitude": False,
            "Longitude": False
        },
        color="Status",
        color_discrete_map={
            "Operational": "#41b349",
            "Not Operational": "#f03752",
            "Closed":"#2d0c13",
            "Closed for Construction":"#fbc82f",
        },
        zoom=zoom,
        center={"lat": center_lat, "lon": center_lon},
        height=height
    )
    
    # Set map style
    fig.update_layout(
        mapbox_style="carto-positron",
        margin={"r": 0, "t": 0, "l": 0, "b": 0},
        # legend_title_text="Restroom Status",
    )

    fig.update_layout(
        hoverlabel=dict(
            font_family="Georgia, Palatino, serif",
            font_size=12,
            # font_color= "#AE3737",
            # # bgcolor="white",
            # bordercolor="gray"
        )
    )

    
    # Add responsive layout and interactive controls
    fig.update_layout(
        autosize=True,
        hovermode='closest',
        # clickmode='event+select',
        dragmode='pan',
        showlegend=True,
    )

    fig.update_traces(marker=dict(size=5))  # Set all points to 5 pixels in size
    
    return fig

def save_map_to_html(fig, output_file="nyc_public_restrooms_map.html"):
    """
    Save map as HTML file
    """
    fig.write_html(
        output_file,         
        config={
            'scrollZoom': True,
            'displayModeBar': True,
            'modeBarButtonsToRemove': ['select2d', 'lasso2d'],
            'responsive': True
        },
        include_plotlyjs='cdn'
    )
    

def main(data_path='./data/Public_Restrooms.csv'):
    """
    Main function, only calls create map and save map functions
    
    Args:
        data_path (str): Path to data file
    """
    # Create map
    fig = create_restroom_map(data_path)
    
    # Display map
    fig.show()
    
    # Save map
    save_map_to_html(fig)

# If run as main program
# if __name__ == "__main__":
#     DATA_PATH = './data/Public_Restrooms.csv'
#     main(DATA_PATH)

In [ ]:
#        hexbin_ploty.py           
#####################################
import pandas as pd
import geopandas as gpd
from shapely import wkt
import plotly.express as px
import plotly.figure_factory as ff
import numpy as np
from shapely.geometry import Point, LineString, MultiLineString, Polygon
import h3
import json
from shapely.ops import transform
import pyproj
from functools import partial


def load_data(file_path):
    """Load CSV data and process geographic information"""
    # Read CSV file
    df = pd.read_csv(file_path)
    
    # Convert WKT format to Shapely geometry objects
    df['geometry'] = df['the_geom'].apply(wkt.loads)
    
    # Create GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")
    
    # Calculate length of each street (meters)
    gdf['length_m'] = gdf.to_crs(epsg=3857).geometry.length
    
    return gdf


def extract_points_from_linestrings(gdf):
    """
    Extract points from line data for generating hexbin maps
    Extracts points proportionally to line length, ensuring longer segments contribute more points
    
    Args:
        gdf: GeoDataFrame containing line data
        
    Returns:
        DataFrame: Contains extracted points and their corresponding intensity values
    """
    points = []
    
    for idx, row in gdf.iterrows():
        # Invert Rank value so smaller Ranks correspond to higher intensity
        intensity = 6 - row['Rank']  # Convert 0-5 range to 6-1
        
        geom = row.geometry
        # Determine number of points to extract based on length
        # Longer lengths extract more points, increasing their weight
        n_points = max(3, int(row['length_m'] / 50))  # One point per 50m, minimum 3 points
        
        if isinstance(geom, MultiLineString):
            for line in geom.geoms:
                # Sample points uniformly from the line
                for i in range(n_points):
                    point = line.interpolate(i / (n_points - 1), normalized=True)
                    points.append({
                        'lon': point.x,
                        'lat': point.y,
                        'intensity': intensity,
                        'street_id': row['segmentid']
                    })
        else:
            # Sample points uniformly from the line
            for i in range(n_points):
                point = geom.interpolate(i / (n_points - 1), normalized=True)
                points.append({
                    'lon': point.x,
                    'lat': point.y,
                    'intensity': intensity,
                    'street_id': row['segmentid']
                })
    
    return pd.DataFrame(points)


def create_hexbin_mapbox(points_df, center=None, zoom=10, hex_size=45, 
                         mapbox_token=None, style='carto-positron'):
    """
    Create interactive Plotly Hexbin map
    
    Args:
        points_df: DataFrame containing point data (must include 'lat', 'lon', 'intensity' columns)
        center: Map center point [lat, lon]
        zoom: Initial zoom level
        hex_size: Hexagon size
        mapbox_token: Mapbox access token (optional)
        style: Map style
        
    Returns:
        plotly.graph_objects.Figure: Plotly figure object
    """
    if center is None:
        # Default to data center as map center
        center = {
            'lat': points_df['lat'].mean(),
            'lon': points_df['lon'].mean()
        }
    
    # Create hexbin map
    fig = ff.create_hexbin_mapbox(
        data_frame=points_df, 
        lat='lat', 
        lon='lon',
        color='intensity',
        nx_hexagon=hex_size,
        opacity=0.2,
        labels={'color': 'Intensity'},
        color_continuous_scale='Viridis_r',  # 'Plasma', 'Inferno', 'Magma', 'Cividis'
        mapbox_style=style,
        min_count=1,  # Minimum 1 point needed to display hexagon
        show_original_data=False,  # Don't show original points
        original_data_marker=dict(opacity=0.1, size=2, color='black'),
        center=center,
        zoom=zoom,
    )

    # Set figure layout
    fig.update_layout(
        # title='NYC Pedestrian Intensity Map',
        autosize=True,
        height=800,
        margin=dict(t=50, b=0, l=0, r=0),
        mapbox=dict(
            accesstoken=mapbox_token,
            style=style
        ),
        showlegend=True,
        hoverlabel=dict(
            font_family="Georgia, Palatino, serif"  # Added custom font for hover tooltip
        )
    )
    fig.update_layout(
        hoverlabel=dict(
            font_family="Georgia, Palatino, serif",  # Already added font settings
            bgcolor="white",     # Set background color
            bordercolor="black", # Set border color
            font_color="black"   # Set text color
        )
    )

    # Add hover tooltip - note the format is preserved but with custom font
    fig.update_traces(
        hovertemplate='<b>Intensity</b>: %{z:.2f}<br><extra></extra>'
    )

    return fig


def create_h3_hexbin(gdf, resolution=9):
    """
    Create high-precision hexagon map using H3 indexing system
    
    Args:
        gdf: GeoDataFrame containing street data
        resolution: H3 index resolution (7-10)
        
    Returns:
        GeoDataFrame: Contains hexagons and their intensity values
    """
    # Extract all points
    points_df = extract_points_from_linestrings(gdf)
    
    # Create H3 index for each point
    points_df['h3_index'] = points_df.apply(
        lambda row: h3.geo_to_h3(row['lat'], row['lon'], resolution), 
        axis=1
    )
    
    # Group by H3 index and calculate average intensity
    hex_data = points_df.groupby('h3_index').agg({
        'intensity': 'mean',
        'street_id': 'count',
        'lat': 'mean',
        'lon': 'mean'
    }).reset_index()
    
    hex_data.rename(columns={'street_id': 'point_count'}, inplace=True)
    
    # Convert H3 index to hexagon geometry
    hex_data['geometry'] = hex_data['h3_index'].apply(
        lambda h: Polygon(h3.h3_to_geo_boundary(h, geo_json=True))
    )
    
    # Create GeoDataFrame
    hex_gdf = gpd.GeoDataFrame(hex_data, geometry='geometry', crs='EPSG:4326')
    
    return hex_gdf


def create_pedastrain_intensity(file_path, resolution=9, hex_size=30):
    """
    Main function: Load data and generate interactive map
    
    Args:
        file_path: Path to CSV file
        resolution: H3 resolution (optional)
        hex_size: Plotly hexagon size (optional)
        
    Returns:
        plotly.graph_objects.Figure: Interactive map
    """
    # Load data
    print("Loading data...")
    gdf = load_data(file_path)
    print(f"Loaded {len(gdf)} street records")
    
    # Extract points
    print("Processing geometry data...")
    points_df = extract_points_from_linestrings(gdf)
    print(f"Extracted {len(points_df)} points from line segments")
    
    # Create Hexbin map
    print("Creating interactive map...")
    fig = create_hexbin_mapbox(
        points_df,
        center={'lat': 40.7128, 'lon': -74.0060},  # NYC center
        zoom=10,
        hex_size=hex_size
    )

    fig.update_traces(marker=dict(line=dict(width=0)))
    fig.update_layout(coloraxis_showscale=False)

    
    return fig


def save_and_show_map(fig, output_file='nyc_sidewalk_intensity_hexbin.html'):
    """Save and display map"""
    fig.write_html(
        output_file,
        config={
            'scrollZoom': True,
            'displayModeBar': True,
            'modeBarButtonsToRemove': ['select2d', 'lasso2d'],
            'responsive': True
        },
        include_plotlyjs='cdn'
    )
    print(f"Map saved to {output_file}")
    return fig

# Ensure wheel zoom is enabled when saving

# if __name__ == "__main__":
#     # Replace with your data file path
#     DATA_PATH = "./data/"
#     file_path = DATA_PATH + "Pedestrian_Mobility_Plan_Pedestrian_Demand_20250425.csv"
    
#     # Create map
#     fig = create_pedastrain_intensity(file_path, hex_size=45)
    
#     # Save and display map
#     save_and_show_map(fig)
    
#     print("NYC pedestrian intensity hexbin map successfully created!")


In [ ]:
#       refactor_integrate.py           
#####################################
import pandas as pd
import plotly.graph_objects as go

from hexbin_ploty import create_pedastrain_intensity
from toilet_scatter import create_restroom_map

DATA_PATH = "./data/"

path_pedastrain = DATA_PATH + "Pedestrian_Mobility_Plan_Pedestrian_Demand_20250425.csv"
path_toilet = DATA_PATH + "Public_Restrooms.csv"

fig1 = create_pedastrain_intensity(path_pedastrain)
fig2 = create_restroom_map(path_toilet)

# Create overlapping layers
fig = go.Figure()

# Add a virtual trace to control the pedestrian intensity layer
fig.add_trace(go.Scatter(
    x=[40.7128],  # Use map center coordinates
    y=[-74.0060],
    mode='markers',
    marker=dict(size=1, color='rgba(0,0,0,0)'),  # Completely transparent
    name="Show <b>Pedestrian intensity</b>",
    legendgroup="pedestrian",
    showlegend=True
))

# Add all traces from fig1 to new layer (pedestrian intensity)
for trace in fig1.data:
    trace.name = "Pedestrian Intensity"  # Add name for legend
    trace.legendgroup = "pedestrian"  # Group
    trace.showlegend = False  # Don't show in legend
    fig.add_trace(trace)

# Add all traces from fig2 to new layer (public restrooms)
for trace in fig2.data:
    fig.add_trace(trace)

# Update layout, show legend in top left
fig.update_layout(
    mapbox=dict(
        center=dict(lat=40.7128, lon=-74.0060),  # NYC center
        zoom=10,
        style="carto-positron"
    ),
    coloraxis=dict(
        colorscale='Viridis_r',
        showscale=False,  # Show color bar
        colorbar=dict(
            title="Pedestrian Intensity",
            x=0.01,  # Horizontal position, left
            y=0.5,   # Vertical position, middle
            xanchor="left",
            thickness=20,
            len=0.5   # Color bar length is half the height of the figure
        )
    ),
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
    autosize=True,
    hovermode='closest',
    dragmode='zoom',
    showlegend=True,  # Show legend
    legend=dict(
        x=0.01,  # Horizontal position, left
        y=0.99,  # Vertical position, top
        xanchor="left",  # Left alignment
        yanchor="top",   # Top alignment
        bgcolor="rgba(252, 245, 237, 0.5)",  # Semi-transparent white background
        bordercolor="rgba(0, 0, 0, 0.5)",    # Border color
        font=dict(
            family=" Georgia, Palatino, serif",
            size=12,
            color='rgba(147, 61, 63, 1)',  # Use the same color value
            weight="normal"
        ),
    ),
    # Explicitly hide x and y axes
    xaxis=dict(visible=False),
    yaxis=dict(visible=False)
)

fig.update_layout(
    hoverlabel=dict(
        font_family="Georgia, Palatino, serif",
        font_size=12,
    )
)

# Ensure wheel zoom is enabled when saving
fig.write_html(
    "nyc_scatter_hexbin.html",
    config={
        'scrollZoom': True,
        'displayModeBar': False,
        'modeBarButtonsToRemove': ['select2d', 'lasso2d'],
        'responsive': True
    },
    include_plotlyjs='cdn'
)

# Display layers
# fig.show()

In [ ]:
#       refactor_range_dynamic.py           
#####################################
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import re

def load_restroom_data(file_path):
    """
    Load public restroom data
    
    Args:
        file_path (str): Path to CSV file
    
    Returns:
        pandas.DataFrame: Loaded data
    """
    df = pd.read_csv(file_path)
    return df

def meters_to_pixels_for_plotly(lat, radius_meters):
    """
    Calculate pixel size for Plotly map for given latitude and radius (meters)
    
    Args:
        lat (float): Latitude
        radius_meters (float): Radius in meters
    
    Returns:
        float: Plotly marker size
    """
    # Latitude adjustment factor - higher latitude, larger scaling in longitude direction
    lat_factor = 1 / np.cos(np.radians(abs(lat)))
    
    # Convert meters to Plotly marker.size
    # Using empirical scale factor, can be adjusted based on visual effect
    scale_factor = 20
    
    # Apply latitude adjustment and scale conversion
    marker_size = (radius_meters / scale_factor) * np.sqrt(lat_factor)
    
    return marker_size

def create_restroom_map_with_coverage(data_path, initial_radius=500, min_radius=300, max_radius=1000, radius_step=100, center_lat=40.7128, center_lon=-74.0060, zoom=10, height=600):
    """
    Create public restroom map visualization with coverage radius
    
    Args:
        data_path (str): Path to CSV file
        initial_radius (int): Initial coverage radius (meters)
        min_radius (int): Minimum coverage radius value (meters)
        max_radius (int): Maximum coverage radius value (meters)
        radius_step (int): Radius slider step size (meters)
        center_lat (float): Map center latitude
        center_lon (float): Map center longitude
        zoom (int): Map zoom level
        height (int): Map height
    
    Returns:
        plotly.graph_objects.Figure: Created map figure
    """
    # Load data
    df = load_restroom_data(data_path)
    
    # Create base map - initially only with restroom locations
    fig = px.scatter_mapbox(
        df, 
        lat="Latitude", 
        lon="Longitude",
        hover_name="Facility Name",
        hover_data={
            "Location Type": True,
            "Status": True,
            "Hours of Operation": False,
            "Accessibility": True,
            "Latitude": False,
            "Longitude": False
        },
        color="Status",
        color_discrete_map={
            "Operational": "#41b349",
            "Not Operational": "#f03752",
            "Closed":"#2d0c13",
            "Closed for Construction":"#fbc82f",
        },
        zoom=zoom,
        center={"lat": center_lat, "lon": center_lon},
        # height=height
    )
    
    # Set map style
    fig.update_layout(
        mapbox_style="carto-positron",
        margin={"r": 0, "t": 0, "l": 0, "b": 0},  # Modify top and bottom margins to accommodate slider
    )
    
    # Set point size
    fig.update_traces(marker=dict(size=5))
    
    # Calculate average latitude for radius conversion
    avg_lat = df["Latitude"].mean()
    
    # Calculate marker size for coverage radius
    marker_size = meters_to_pixels_for_plotly(avg_lat, initial_radius)
    
    # Add semi-transparent circular coverage areas as new layer
    fig.add_trace(
        go.Scattermapbox(
            lat=df["Latitude"],
            lon=df["Longitude"],
            mode="markers",
            marker=dict(
                size=marker_size,  # Use calculated radius size
                color="rgba(92, 30, 25, 0.5)",  
                opacity=0.5,
            ),
            name=f"Coverage Range ({initial_radius} meters)",
            showlegend=False,
            hoverinfo="none",  # Disable hover display to avoid duplication with original points
        )
    )
    
    # Create list of values for slider range
    radius_values = list(range(min_radius, max_radius + 1, radius_step))
    
    # For display purposes
    label_radius_values = [300, 500, 700, 900]
    # Create slider for static HTML
    sliders = [dict(
        active=radius_values.index(initial_radius) if initial_radius in radius_values else 0,  # Default slider position
        currentvalue={"prefix": "Range: ", "suffix": " meters", "visible": True, "font": {"size": 12, "color": "#933D3F"},},
        pad={"t": 5, "l": 5},  # Reduce top padding, increase left padding
        len=0.2,  # Slider length, between 0-1, 1 is full width
        x=0.01,   # Slider left position, between 0-1
        y=0.98,   # Slider top position, between 0-1
        yanchor="top",  # Top alignment
        xanchor="left", # Left alignment

        font={ "family": "Georgia, Palatino, serif", "color": "#933D3F", "size": 10},
        bordercolor="#933D3F",
        activebgcolor="#933D3F",

        steps=[
            dict(
                method="restyle",
                args=[
                    {"marker.size": meters_to_pixels_for_plotly(avg_lat, radius)},
                    [4]  # Only update the second trace (coverage circles) marker.size
                ],
                # label=str(radius) if radius in label_radius_values else "",
                label=f"{radius}m",
                value=radius,
            ) for radius in radius_values
        ]
    )]
    
    # Update layout to include slider
    fig.update_layout(
        sliders=sliders,
        # title=f"NYC Public Restroom Distribution (Coverage Radius: {initial_radius} meters)",
        autosize=True,
        hovermode='closest',
        clickmode='event+select',
        dragmode='pan',
        showlegend=False,
    )
    
    return fig

def save_map_to_html(fig, output_file="nyc_coverage.html", initial_zoom=10, initial_radius=500):
    """
    Save map as HTML file with zoom adjustment script
    
    Args:
        fig (plotly.graph_objects.Figure): Map figure
        output_file (str): Output HTML filename
        initial_zoom (int): Initial zoom level
        initial_radius (int): Initial coverage radius (meters)
    """
    # Generate basic HTML
    html_string = fig.to_html(
        config={
            'scrollZoom': True,
            'displayModeBar': False,
            'modeBarButtonsToRemove': ['select2d', 'lasso2d'],
            'responsive': True
        },
        include_plotlyjs='cdn'
    )
    
    # Inject zoom adjustment script
    zoom_adjust_script = """
    <script>
    document.addEventListener('DOMContentLoaded', function() {
        var myPlot = document.getElementById('CHART_ID');
        var initialZoom = INITIAL_ZOOM; // Initial zoom level
        var initialRadius = INITIAL_RADIUS; // Initial radius (meters)
        var currentRadius = initialRadius; // Current radius
        var currentZoom = initialZoom; // Current zoom level
        
        // Add information label
        var infoDiv = document.createElement('div');
        infoDiv.style.position = 'absolute';
        // infoDiv.style.top = '80px';
        infoDiv.style.right = '10px';
        infoDiv.style.backgroundColor = 'transparent';
        infoDiv.style.padding = '5px';
        infoDiv.style.borderRadius = '5px';
        infoDiv.style.zIndex = '1000';
        infoDiv.style.maxWidth = '200px';

        infoDiv.style.fontFamily = 'Georgia, Palatino, serif'; // Set font
        infoDiv.style.fontSize = '12px';               // Font size
        // infoDiv.style.fontWeight = 'bold';             // Bold text
        infoDiv.style.color = '#933D3F';               // Text color

        infoDiv.style.top = '20.5%'; 
        infoDiv.style.left = '2%';

        infoDiv.innerHTML = 'Current Radius: ' + initialRadius + 'meters<br>Zoom Lebel: 10';
        document.body.appendChild(infoDiv);
        
        // Helper function: Calculate actual pixel size for map zoom level
        function calculateMarkerSize(radiusMeters, zoom) {
            // Calculate based on NYC average latitude
            var avgLat = 40.7128;
            var latFactor = 1 / Math.cos(avgLat * Math.PI / 180);
            
            // Scaling ratio for different zoom levels
            var zoomFactor = Math.pow(2, zoom - initialZoom);
            
            // Base scaling factor
            var scaleFactor = 20;
            
            // Calculate final size
            return (radiusMeters / scaleFactor) * Math.sqrt(latFactor) * zoomFactor;
        }
        
        // Helper function: Calculate actual pixel size for map zoom level
        function calculateMarkerSize(radiusMeters, zoom) {
            // Calculate based on NYC average latitude
            var avgLat = 40.7128;
            var latFactor = 1 / Math.cos(avgLat * Math.PI / 180);
            
            // Scaling ratio for different zoom levels
            var zoomFactor = Math.pow(2, zoom - initialZoom);
            
            // Base scaling factor
            var scaleFactor = 20;
            
            // Calculate final size
            return (radiusMeters / scaleFactor) * Math.sqrt(latFactor) * zoomFactor;
        }
        
        // Listen for map zoom and move events
        if (myPlot) {
            myPlot.on('plotly_relayout', function(eventData) {
                // Check if zoom changed
                if (eventData['mapbox.zoom'] !== undefined) {
                    var newZoom = eventData['mapbox.zoom'];
                    
                    // Calculate new marker size
                    var newSize = calculateMarkerSize(currentRadius, newZoom);
                    
                    // Update coverage circle size
                    Plotly.restyle(myPlot, {'marker.size': newSize}, [4]);
                    
                    // Update current zoom level
                    currentZoom = newZoom;
                    
                    // Update info box
                    infoDiv.innerHTML = 'Current Radius: ' + currentRadius + ' meters<br>Zoom Level: ' + newZoom.toFixed(1);
                }
            });
            
            // Listen for slider changes, update current radius
            myPlot.on('plotly_sliderchange', function(e) {
                // Get slider value directly from event
                if (e && e.step !== undefined && e.step.value !== undefined) {
                    currentRadius = parseInt(e.step.value);
                    
                    // Update info box
                    infoDiv.innerHTML = 'Current Radius: ' + currentRadius + ' meters<br>Zoom Level: ' + currentZoom.toFixed(1);
                    
                    // Optional: Update marker size based on new radius and current zoom level
                    var newSize = calculateMarkerSize(currentRadius, currentZoom);
                    Plotly.restyle(myPlot, {'marker.size': newSize}, [4]);
                }
            });
        }
    });
    </script>
    """
    
    
    # Replace chart ID and initial values
    div_id = re.search(r'id="([^"]*)"', html_string).group(1)
    zoom_adjust_script = zoom_adjust_script.replace('CHART_ID', div_id)
    zoom_adjust_script = zoom_adjust_script.replace('INITIAL_ZOOM', str(initial_zoom))
    zoom_adjust_script = zoom_adjust_script.replace('INITIAL_RADIUS', str(initial_radius))
    
    # Insert script before </body>
    html_string = html_string.replace('</body>', zoom_adjust_script + '</body>')
    
    # Write to file
    with open(output_file, 'w') as f:
        f.write(
            html_string,
        )

def main(data_path='./data/Public_Restrooms.csv'):
    """
    Main function
    
    Args:
        data_path (str): Path to data file
    """
    # Set initial parameters
    initial_radius = 300  # Initial radius of 500 meters
    initial_zoom = 10     # Initial zoom level
    
    # Create map with coverage radius
    fig = create_restroom_map_with_coverage(
        data_path,
        initial_radius=initial_radius,
        min_radius=300,       # Minimum radius of 300 meters
        max_radius=900,      # Maximum radius of 900 meters
        radius_step=50,      # Slider step of 200 meters
        zoom=initial_zoom     # Set initial zoom level
    )
    
    # Display map
    fig.show()
    
    # Save map, and pass initial zoom level and radius value for JavaScript
    save_map_to_html(fig, initial_zoom=initial_zoom, initial_radius=initial_radius)

# If run as main program
# if __name__ == "__main__":
#     DATA_PATH = './data/Public_Restrooms.csv'
#     main(DATA_PATH)

In [ ]:
#       affinity_heatmatrix.py           
#####################################
import pandas as pd
import numpy as np
from bokeh.plotting import figure, output_file, save
from bokeh.models import ColumnDataSource, LinearColorMapper, ColorBar, BasicTicker
from bokeh.palettes import Iridescent18
from bokeh.transform import transform  # 添加 transform 导入
from bokeh.layouts import column
from sklearn.metrics.pairwise import cosine_similarity

def load_and_process_data(poi_file, toilet_file):
    """
    Load and process POI and toilet datasets
    """
    # Load POI data
    poi_df = pd.read_csv(poi_file)
    
    # Extract latitude and longitude from the_geom
    poi_df['Longitude'] = poi_df['the_geom'].str.extract(r'POINT \(([-\d.]+)', expand=False).astype(float)
    poi_df['Latitude'] = poi_df['the_geom'].str.extract(r'POINT \([-\d.]+ ([-\d.]+)', expand=False).astype(float)
    
    # Filter out Residential type (FACILITY_T=1) and invalid data
    poi_df = poi_df[poi_df['FACILITY_T'] != 1]
    
    # Ensure valid latitude and longitude data
    poi_df = poi_df.dropna(subset=['Latitude', 'Longitude'])
    poi_df = poi_df[(poi_df['Latitude'] > 40.0) & (poi_df['Latitude'] < 41.0) &
                   (poi_df['Longitude'] > -75.0) & (poi_df['Longitude'] < -73.0)]
    
    # Load toilet data
    toilet_df = pd.read_csv(toilet_file)
    
    # Extract clean latitude and longitude data
    if 'Location' in toilet_df.columns:
        if toilet_df['Location'].dtype == object and toilet_df['Location'].str.contains('POINT').any():
            # If Location column contains POINT format, extract coordinates
            toilet_df['Longitude'] = toilet_df['Location'].str.extract(r'POINT \(([-\d.]+)', expand=False).astype(float)
            toilet_df['Latitude'] = toilet_df['Location'].str.extract(r'POINT \([-\d.]+ ([-\d.]+)', expand=False).astype(float)
    
    # Ensure required columns exist
    expected_cols = ['Latitude', 'Longitude']
    for col in expected_cols:
        if col not in toilet_df.columns:
            raise ValueError(f"Missing {col} column in toilet dataset")
    
    # Facility type name dictionary
    facility_types = {
        2: "Educational", 
        3: "Cultural", 
        4: "Recreational", 
        5: "Social Services",
        6: "Transportation", 
        7: "Commercial", 
        8: "Government", 
        9: "Religious", 
        10: "Health Services", 
        11: "Public Safety", 
        12: "Water Services", 
        13: "Miscellaneous"
    }
    
    # Add facility type names
    poi_df['FACILITY_NAME'] = poi_df['FACILITY_T'].map(facility_types)
    
    # Add type marker for toilet data
    toilet_df['FACILITY_T'] = 14  # Use a non-duplicate number
    toilet_df['FACILITY_NAME'] = "Public Restrooms"
    
    return poi_df, toilet_df

def create_spatial_density_grid(df, lat_min, lat_max, lon_min, lon_max, grid_size=0.01):
    """
    Create spatial density grid for each facility type
    """
    # Create grid
    lat_bins = np.arange(lat_min, lat_max + grid_size, grid_size)
    lon_bins = np.arange(lon_min, lon_max + grid_size, grid_size)
    
    # Initialize result dictionary
    density_map = {}
    
    # Get unique facility types
    facility_types = df['FACILITY_T'].unique()
    
    # Create density grid for each facility type
    for facility_type in facility_types:
        # Filter facilities of this type
        subset = df[df['FACILITY_T'] == facility_type]
        
        # Ensure sufficient data points
        if len(subset) < 2:
            print(f"Warning: Facility type {facility_type} has only {len(subset)} data points, cannot create valid density grid. Skipping.")
            continue
        
        try:
            # Create 2D histogram
            hist, _, _ = np.histogram2d(
                subset['Latitude'], 
                subset['Longitude'], 
                bins=[lat_bins, lon_bins]
            )
            
            # Normalize density to balance differences in facility counts
            if hist.sum() > 0:  # Avoid division by zero
                hist = hist / hist.sum()
                
            # Flatten histogram as feature vector
            density_map[facility_type] = hist.flatten()
        except Exception as e:
            print(f"Error processing facility type {facility_type}: {e}")
    
    return density_map

def compute_similarity_matrix(density_map):
    """
    Compute cosine similarity matrix between facility types
    """
    # Get all facility types
    facility_types = list(density_map.keys())
    
    # Create feature matrix, each row is a density vector for a facility type
    features = np.array([density_map[ft] for ft in facility_types])
    
    # Compute cosine similarity
    similarity_matrix = cosine_similarity(features)
    
    return similarity_matrix, facility_types




def create_heatmap(similarity_matrix, facility_names, output_file_path="nyc_spatial_similarity_heatmap.html"):
    """
    Create a Bokeh heatmap visualization of the similarity matrix
    """
    # Prepare data for bokeh
    facility_count = len(facility_names)
    
    # Create a DataFrame for the heatmap
    heatmap_data = {
        'facility1': [],
        'facility2': [],
        'x': [],
        'y': [],
        'similarity': []
    }
    
    for i in range(facility_count):
        for j in range(facility_count):
            
            heatmap_data['facility1'].append(facility_names[i])
            heatmap_data['facility2'].append(facility_names[j])
            heatmap_data['x'].append(facility_names[j])
            heatmap_data['y'].append(facility_names[i])  
            heatmap_data['similarity'].append(similarity_matrix[i, j])
        
    source = ColumnDataSource(data=heatmap_data)
    
    from bokeh.palettes import Iridescent18
    colors = list(Iridescent18)
    colors = colors[0:15]
    mapper = LinearColorMapper(
        palette=colors, 
        low=0, 
        high=1
    )
    # Create color mapper
    mapper = LinearColorMapper(
        palette=colors, 
        low=0, 
        high=1
    )
    
    TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"
    
    # Set up figure 
    p = figure(
        width=700,
        height=700,
        x_range=facility_names,  
        y_range=facility_names,  
        toolbar_location=None,
        x_axis_location="below",  
        tooltips=[('Facilities', '@facility1 - @facility2'), ('Similarity', '@similarity{0.000}')],
        background_fill_color=None,
        border_fill_color=None,
        outline_line_color=None,
        tools=TOOLS,
        sizing_mode='stretch_both' 
    )
    
    p.rect(
        x='x',
        y='y',
        width=1.0,
        height=1.0,
        source=source,
        fill_color={'field': 'similarity', 'transform': mapper},
        line_color=None,
        alpha=0.8
    )
    
    color_bar = ColorBar(
        color_mapper=mapper,
        ticker=BasicTicker(desired_num_ticks=7),
        label_standoff=6,
        border_line_color=None,
        location=(0, 0),
        width=20,
        height=300,
        title="",
        background_fill_color=None,
        visible=False 
    )
    
    p.add_layout(color_bar, 'right')
    
    # Set axes style
    p.axis.axis_line_color = None
    p.axis.major_tick_line_color = None
    p.axis.major_label_text_font_size = "10px"
    p.axis.major_label_standoff = 0
    p.xaxis.major_label_orientation = 0.9  
    p.yaxis.major_label_orientation = 0 
    
    # Remove grid lines
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None

    p.axis.major_label_text_font = "Georgia, Palatino, serif"
    p.axis.major_label_text_font_style = "normal"

    
    # Output to file
    output_file(output_file_path)
    save(p)
    
    print(f"Heatmap saved to {output_file_path}")
    return p

def main(poi_file, toilet_file, output_file="nyc_spatial_similarity_heatmap.html"):
    """
    Main function for the analysis
    """
    # 1. Load and process data
    poi_df, toilet_df = load_and_process_data(poi_file, toilet_file)
    
    # 2. Combine datasets
    combined_df = pd.concat([poi_df[['FACILITY_T', 'FACILITY_NAME', 'Latitude', 'Longitude']], 
                            toilet_df[['FACILITY_T', 'FACILITY_NAME', 'Latitude', 'Longitude']]])
    
    # 3. Set NYC boundaries
    nyc_bounds = {
        'lat_min': 40.5,
        'lat_max': 40.9,
        'lon_min': -74.1,
        'lon_max': -73.7
    }
    
    # 4. Create spatial density grid
    density_map = create_spatial_density_grid(
        combined_df, 
        nyc_bounds['lat_min'], 
        nyc_bounds['lat_max'], 
        nyc_bounds['lon_min'], 
        nyc_bounds['lon_max']
    )
    
    # 5. Compute similarity matrix
    similarity_matrix, facility_types = compute_similarity_matrix(density_map)
    
    # Get facility type names
    facility_names = []
    for ft in facility_types:
        if ft == 14:  # Toilet code
            facility_names.append("Public Restrooms")
        else:
            name = combined_df[combined_df['FACILITY_T'] == ft]['FACILITY_NAME'].iloc[0]
            facility_names.append(name)
    
    # 6. Create heatmap
    create_heatmap(similarity_matrix, facility_names, output_file)
    
    # 7. Analyze similarity with toilets
    toilet_index = facility_types.index(14)  # Index of toilets
    similarity_with_toilet = similarity_matrix[toilet_index]
    
    # Create similarity ranking
    similarity_ranking = [(name, sim) for name, sim in zip(facility_names, similarity_with_toilet)]
    similarity_ranking.sort(key=lambda x: x[1], reverse=True)
    
    # Return similarity ranking (excluding toilets themselves)
    return [(name, sim) for name, sim in similarity_ranking if name != "Public Restrooms"]

if __name__ == "__main__":
    # File path parameters
    poi_file = "./data/Points_of_Interest_20250425.csv"
    toilet_file = "./data/Public_Restrooms.csv"
    
    # Run analysis
    similarity_ranking = main(poi_file, toilet_file)
    
    # Print facility types with spatial distribution most similar to public restrooms
    print("Facility types with spatial distribution most similar to public restrooms:")
    for name, similarity in similarity_ranking:
        print(f"{name}: {similarity:.4f}")

In [ ]:
#       time_solar_final.py           
#####################################
import pandas as pd
import numpy as np
import re
import json
from collections import defaultdict

# Color mapping
COLOR_MAP = {
    'NYC Parks': '#8BC34A',           # Green
    'NYPL': '#64B5F6',                # Blue
    'NYC Parks Concessionaire': '#FFB74D',  # Orange
    'QPL': '#BA68C8',                 # Purple
    'BPL': '#EF9A9A',                 # Red
    'Others': '#90A4AE'               # Gray
}

# Main operator list
MAIN_OPERATORS = ['NYC Parks', 'NYPL', 'NYC Parks Concessionaire', 'QPL', 'BPL', 'Others']

# Days of week
DAYS_OF_WEEK = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

def load_data(csv_file):
    """Load CSV data"""
    return pd.read_csv(csv_file)

def parse_time_ranges(time_str):
    """Parse time range strings into hour lists"""
    if pd.isna(time_str) or time_str == 'CLOSED':
        return []
    
    if time_str == '24 HOURS':
        return list(range(24))
    
    hours = []
    time_ranges = time_str.split(',')
    
    for time_range in time_ranges:
        match = re.match(r'(\d{1,2}):(\d{2})-(\d{1,2}):(\d{2})', time_range.strip())
        if match:
            start_hour, start_min, end_hour, end_min = map(int, match.groups())
            
            # Convert to 24-hour format
            if 'PM' in time_range and start_hour < 12:
                start_hour += 12
            if 'PM' in time_range and end_hour < 12:
                end_hour += 12
            
            # Add all hour segments
            current_hour = start_hour
            while current_hour != end_hour:
                hours.append(current_hour)
                current_hour = (current_hour + 1) % 24
    
    return hours

def categorize_operator(operator):
    """Categorize operator into a main category or 'Others'"""
    return operator if operator in MAIN_OPERATORS[:-1] else 'Others'

def calculate_hourly_counts(df, day):
    """Calculate facility count for each operator by hour for a specific day"""
    hourly_counts = {hour: defaultdict(list) for hour in range(24)}
    
    for _, row in df.iterrows():
        if pd.isna(row[day]) or row[day] == '':
            continue
            
        hours = parse_time_ranges(row[day])
        operator = categorize_operator(row['Operator'])
        
        for hour in hours:
            hourly_counts[hour][operator].append(row['Facility Name'])
    
    return hourly_counts

def calculate_global_max_count(df):
    """Calculate global maximum count across all days"""
    global_max = 0
    
    for day in DAYS_OF_WEEK:
        hourly_counts = calculate_hourly_counts(df, day)
        
        # Calculate AM and PM maximums, then get global maximum
        am_max = max(sum(len(hourly_counts[h][op]) for op in hourly_counts[h]) for h in range(12))
        pm_max = max(sum(len(hourly_counts[h][op]) for op in hourly_counts[h]) for h in range(12, 24))
        day_max = max(am_max, pm_max)
        
        global_max = max(global_max, day_max)
    
    return global_max

def generate_all_days_data(df):
    """Generate data for all days (for client-side rendering)"""
    global_max_count = calculate_global_max_count(df)
    all_data = {"global_max_count": global_max_count}
    
    for day in DAYS_OF_WEEK:
        hourly_counts = calculate_hourly_counts(df, day)
        
        day_data = {
            "day": day,
            "hourly_data": {}
        }
        
        for hour in range(24):
            operators_data = {}
            for operator, facilities in hourly_counts[hour].items():
                operators_data[operator] = {
                    "count": len(facilities),
                    "facilities": facilities
                }
            day_data["hourly_data"][hour] = operators_data
            
        all_data[day] = day_data
    
    return all_data

def create_nyc_bathroom_visualization(csv_file='./data/Public_Restrooms_time.csv'):
    """Main function, create NYC bathroom visualization"""
    # Load data
    df = load_data(csv_file)
    
    # Generate data for all days
    all_data = generate_all_days_data(df)
    all_data_json = json.dumps(all_data)
    
    # Generate HTML content
    html_content = f"""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>NYC Public Bathroom Hours</title>
        <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
        <style>
            body {{
                font-family: "Georgia, Palatino, serif";
                margin: 0;
                background-color: transparent;
            }}
            .container {{
                display: flex;
                flex-direction: column;
                align-items: center;
                max-width: 1200px;
                background-color: #FCF5ED;
            }}
            h1 {{
                color: #333;
                text-align: center;
                margin-bottom: 20px;
            }}
            .subtitle {{
                text-align: center;
                color: #666;
                margin-top: -10px;
                margin-bottom: 20px;
            }}
            .day-selector {{
                margin: 20px 0;
                text-align: center;
                width: 100%;
            }}
            select {{
                padding: 10px 15px;
                font-size: 12px;
                font-family: "Georgia, Palatino, serif";
                border-radius: 4px;
                border: 1px solid #ccc;
                background-color: white;
                cursor: pointer;
                min-width: 100px;
            }}
            select:focus {{
                outline: none;
                border-color: #4a90e2;
                box-shadow: 0 0 5px rgba(74,144,226,0.5);
            }}
            label[for="day-select"] {{
                font-family: "Georgia, Palatino, serif";
                font-size: 14px;
                font-weight: 500;
                color: #333;
                margin-right: 10px;
            }}
            #chart-container {{
                width: 100%;
                position: relative;
                height: 600px;
            }}
            .subplot-title {{
                font-family: "Georgia, Palatino, serif";
                font-size: 16px;
                font-weight: 600;
                text-align: center;
                color: #333;
                margin-bottom: 5px;
                position: absolute;
                width: 100%;
                top: 10px;
                z-index: 10;
            }}
            .am-title {{
                left: 0;
                width: 50%;
            }}
            .pm-title {{
                right: 0;
                width: 50%;
            }}
            .legend-note {{
                margin-top: 20px;
                text-align: center;
                font-size: 14px;
                color: #666;
            }}
            #chart {{
                width: 100%;
                height: 100%;
            }}
            
            
        </style>
    </head>
    <body>
        <div class="container">
            <div id="chart-container">
                <div id="chart"></div>
            </div>
            
            <div class="day-selector">
                <label for="day-select">Select Day: </label>
                <select id="day-select">
                    {"".join([f'<option value="{day}">{day}</option>' for day in DAYS_OF_WEEK])}
                </select>
            </div>
        </div>
        
        <script>
            // Store all chart data
            const allData = {all_data_json};
            
            // Operator colors
            const operatorColors = {json.dumps(COLOR_MAP)};
            
            // Main operators
            const mainOperators = {json.dumps(MAIN_OPERATORS)};
            
            // Get DOM elements
            const daySelect = document.getElementById('day-select');
            const chartDiv = document.getElementById('chart');
            
            // Check if device is mobile
            function isMobile() {{
                return window.innerWidth <= 768;
            }}
            
            // Get polar chart configuration
            function getPolarConfig(period, globalMaxCount) {{
                return {{
                    hole: 0.05,
                    radialaxis: {{
                        showticklabels: false,
                        visible: false,
                        range: [0, globalMaxCount * 1.1],
                        showline: false,
                        showgrid: false,
                    }},
                    angularaxis: {{
                        showline: false,
                        showgrid: false,
                        ticks: "",
                        ticklen: 5,
                        direction: "clockwise",
                        period: 12,
                        tickmode: "array",
                        tickvals: [0, 30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330],
                        ticktext: period === 'AM' ? 
                                 ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11'] : 
                                 ['12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23'],
                        tickfont: {{ size: 18,  family: "Georgia, Palatino, serif", }}
                    }},
                    bgcolor: "rgba(0,0,0,0)",
                }};
            }}
            
            // Update layout based on screen size
            function updateResponsiveLayout(layout) {{
                const mobile = isMobile();
                
                if (mobile) {{
                    // Mobile layout - stacked vertically
                    layout.grid = {{
                        rows: 2,
                        columns: 1,
                        pattern: 'independent',
                        roworder: 'top to bottom'
                    }};
                    layout.height = 900;
                    
                    // Update polar domains
                    layout.polar.domain = {{
                        x: [0.1, 0.9],
                        y: [0.55, 0.95]
                    }};
                    layout.polar2.domain = {{
                        x: [0.55, 1],
                        y: [0, 1]
                    }};
                }} else {{
                    // Desktop layout - side by side
                    layout.grid = {{
                        rows: 1,
                        columns: 2,
                        pattern: 'independent'
                    }};
                    layout.height = 600;
                    
                    // Update polar domains
                    layout.polar.domain = {{
                        x: [0, 0.43],
                        y: [0, 1]
                    }};
                    layout.polar2.domain = {{
                        x: [0.57, 1],
                        y: [0, 1]
                    }};
                }}
                
                return layout;
            }}
            
            // Create chart based on data
            function createChart(dayData) {{
                const day = dayData.day;
                const hourlyData = dayData.hourly_data;
                
                // AM and PM hours
                const amHours = Array.from({{length: 12}}, (_, i) => i);
                const pmHours = Array.from({{length: 12}}, (_, i) => i + 12);
                
                // Use global maximum count for scaling
                const globalMaxCount = allData.global_max_count;
                
                // Prepare chart data structure
                const traces = [];
                
                // Create stacked data for each period (AM/PM) for each operator
                ['AM', 'PM'].forEach((period, periodIdx) => {{
                    const hours = period === 'AM' ? amHours : pmHours;
                    const subplot = period === 'AM' ? 'polar' : 'polar2';
                    
                    // Adjust angles to match clock layout
                    const theta = hours.map(hour => 
                        ((period === 'AM' ? hour : hour - 12) * 30) + 15
                    );
                    
                    // Prepare stacked data
                    const cumulativeR = {{}};
                    hours.forEach(hour => {{ cumulativeR[hour] = 0; }});
                    
                    // Create a trace for each operator
                    mainOperators.forEach(operator => {{
                        const rValues = [];
                        const textValues = [];
                        
                        hours.forEach(hour => {{
                            const operatorData = (hourlyData[hour] || {{}})[operator] || {{ count: 0, facilities: [] }};
                            const count = operatorData.count;
                            const facilities = operatorData.facilities;
                            
                            rValues.push(count);
                            
                            // Prepare hover text
                            let facilityNames = facilities.slice(0, 5).join('<br>');
                            if (facilities.length > 5) {{
                                facilityNames += `<br>...and ${{facilities.length - 5}} more`;
                            }}
                            
                            let text = `Hour: ${{hour}}:00-${{(hour+1) % 24}}:00<br>Operator: ${{operator}}<br>Count: ${{count}}`;
                            if (count < 0) {{
                                text += `<br><br>Facilities:<br>${{facilityNames}}`;
                            }}
                            textValues.push(text);
                        }});
                        
                        // Create base values for stacking
                        const baseValues = hours.map(hour => cumulativeR[hour]);
                        
                        // Add trace
                        traces.push({{
                            width: 30,
                            type: 'barpolar',
                            r: rValues,
                            theta: theta,
                            base: baseValues,
                            name: operator,
                            marker: {{color: operatorColors[operator]}},
                            hoverinfo: 'text',
                            hovertext: textValues,
                            showlegend: periodIdx === 0, // Only show legend for first period
                            subplot: subplot,
                            legendgroup: operator // Link AM and PM traces in legend
                        }});
                        
                        // Update cumulative height
                        hours.forEach((hour, i) => {{
                            cumulativeR[hour] += rValues[i];
                        }});
                    }});
                }});
                
                // Create layout
                const layout = {{
                    font: {{
                        family: "Georgia, Palatino, serif",
                    }},
                    hoverlabel:{{
                        font: {{
                            family: "Georgia, Palatino, serif",
                        }},
                    }},
                    polar: getPolarConfig('AM', globalMaxCount),
                    polar2: getPolarConfig('PM', globalMaxCount),
                    showlegend: true,
                    paper_bgcolor: "rgba(0,0,0,0)",
                    plot_bgcolor: "rgba(0,0,0,0)",
                    // Add hover label style
                    legend: {{
                        orientation: "h",
                        yanchor: "top",
                        y: -0.15,
                        xanchor: "center",
                        x: 0.5,
                        font: {{
                            family: "Georgia, Palatino, serif",
                            size: 12,
                        }},
                    }},
                    margin: {{t: 45, b: 50}},
                    height: 600,
                    uirevision: 'true' // Maintain legend state
                }};
                
                // Apply responsive layout
                const updatedLayout = updateResponsiveLayout(layout);
                
                // Clear old chart and create new one
                Plotly.react(chartDiv, traces, updatedLayout, {{
                    displayModeBar: false, // Hide mode bar for cleaner look
                    responsive: false,
                    autosize: true,
                }});
            }}
            
            // Handler function for day selection change
            function updateChart() {{
                const selectedDay = daySelect.value;
                const dayData = allData[selectedDay];
                
                if (dayData) {{
                    createChart(dayData);
                }}
            }}
            
            // Set up event listeners on page load
            document.addEventListener('DOMContentLoaded', function() {{
                // Add day selector event listener
                daySelect.addEventListener('change', updateChart);
                
                // Add window resize listener
                window.addEventListener('resize', function() {{
                    const selectedDay = daySelect.value;
                    const dayData = allData[selectedDay];
                    if (dayData) {{
                        createChart(dayData);
                    }}
                }});
                
                // Initialize chart - use currently selected day
                const selectedDay = daySelect.value;
                const dayData = allData[selectedDay];
                if (dayData) {{
                    createChart(dayData);
                }}
            }});
        </script>
    </body>
    </html>
    """
    
    # Save as HTML file
    with open('nyc_clock.html', 'w', encoding='utf-8') as f:
        f.write(html_content)
    
    print("Visualization saved as 'nyc_clock.html'")
    return html_content

# Run code
if __name__ == "__main__":
    create_nyc_bathroom_visualization()
